In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - 1-interpolate_primaries.ipynb
# AUTHOR - James Lane
# PROJECT - tng-dfs
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Compute interpolations of primary radial mass profiles.
'''

__author__ = "James Lane"

In [ ]:
# %load ../../src/nb_modules/nb_imports.txt
### Imports

## Basic
import numpy as np
import sys, os, dill as pickle
import pdb, copy

## Matplotlib
import matplotlib as mpl
from matplotlib import pyplot as plt

## Astropy
from astropy import units as apu

## Analysis
import scipy.interpolate

## galpy
from galpy import orbit
from galpy import potential
from galpy import util as gputil

## Project-specific
src_path = 'src/'
while True:
    if os.path.exists(src_path): break
    if os.path.realpath(src_path).split('/')[-1] in ['tng-dfs','/']:
            raise FileNotFoundError('Failed to find src/ directory.')
    src_path = os.path.join('..',src_path)
sys.path.insert(0,src_path)
from tng_dfs import cutout as pcutout
from tng_dfs import densprofile as pdens
from tng_dfs import util as putil

### Notebook setup

%matplotlib inline
plt.style.use(os.path.join(src_path,'mpl/project.mplstyle')) # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Keywords, loading, pathing

In [ ]:
# %load ../../src/nb_modules/nb_setup.txt
# Keywords
cdict = putil.load_config_to_dict()
keywords = ['DATA_DIR','MW_ANALOG_DIR','RO','VO','ZO','LITTLE_H',
            'MW_MASS_RANGE']
data_dir,mw_analog_dir,ro,vo,zo,h,mw_mass_range = \
    putil.parse_config_dict(cdict,keywords)

# MW Analog 
mwsubs,mwsubs_vars = putil.prepare_mwsubs(mw_analog_dir,h=h,
    mw_mass_range=mw_mass_range,return_vars=True,force_mwsubs=False,
    bulge_disk_fraction_cuts=True)

# Figure path
fig_dir = './fig/interpolated_primaries/'
# epsen_fig_dir = '/epsen_data/scr/lane/projects/tng-dfs/figs/notebooks/sample/'
os.makedirs(fig_dir,exist_ok=True)
# os.makedirs(epsen_fig_dir,exist_ok=True)
show_plots = False

# Fitting path
epsen_fitting_dir = '/epsen_data/scr/lane/projects/tng-dfs/fitting/'+\
    'density_profile/spherical_interpolated_potential/'
os.makedirs(epsen_fitting_dir,exist_ok=True)

# Load tree data
tree_primary_filename = os.path.join(mw_analog_dir,
    'major_mergers/tree_primaries.pkl')
with open(tree_primary_filename,'rb') as handle: 
    tree_primaries = pickle.load(handle)
tree_major_mergers_filename = os.path.join(mw_analog_dir,
    'major_mergers/tree_major_mergers.pkl')
with open(tree_major_mergers_filename,'rb') as handle:
    tree_major_mergers = pickle.load(handle)
n_mw = len(tree_primaries)

### Compute the spherical enclosed mass profile of each primary

In [ ]:
verbose = True
n_thin = 100

def galpy_radial_force_from_enclosed_mass(r, menc, ro=ro, vo=vo):
    '''galpy_radial_force_from_enclosed_mass:

    Compute the radial force from an enclosed mass profile in internal galpy 
    units.
    '''
    gmenc = menc/gputil.conversion.mass_in_msol(vo,ro)
    gr = r/ro
    return -gmenc/gr**2

for i in range(n_mw):
    # if i > 0: continue
    if verbose: print(f'Analyzing MW {i+1}/{n_mw}')

    # Get the primary
    primary = tree_primaries[i]
    z0_sid = primary.subfind_id[0]
    n_snap = len(primary.snapnum)
    primary_filename = primary.get_cutout_filename(mw_analog_dir,
        snapnum=primary.snapnum[0])
    co = pcutout.TNGCutout(primary_filename)
    co.center_and_rectify()
    
    print('Loading data...')
    orbs_star = co.get_orbs('stars')
    orbs_dm = co.get_orbs('dm')
    rs_star = orbs_star.r().to_value(apu.kpc)
    rs_dm = orbs_dm.r().to_value(apu.kpc)
    masses_star = co.get_masses('stars').to_value(apu.Msun)
    masses_dm = co.get_masses('dm').to_value(apu.Msun)
    pe_star = co.get_potential_energy('stars').to_value(apu.km**2/apu.s**2)
    pe_dm = co.get_potential_energy('dm').to_value(apu.km**2/apu.s**2)

    rs = np.concatenate([rs_star,rs_dm])
    # rs = rs_star
    masses = np.concatenate([masses_star,masses_dm])
    # masses = masses_star
    pe = np.concatenate([pe_star,pe_dm])
    # pe = pe_star
    sort_idx = np.argsort(rs)
    rs = rs[sort_idx]
    masses = masses[sort_idx]
    pe = pe[sort_idx]
    menc = np.cumsum(masses)

    rs_thin = rs[::n_thin]
    masses_thin = masses[::n_thin]
    pe_thin = pe[::n_thin]
    menc_thin = menc[::n_thin]

    print('Computing interpolated potential...')
    grforce = galpy_radial_force_from_enclosed_mass(rs_thin, menc_thin, ro, vo)
    rforce_interp = scipy.interpolate.interp1d(rs_thin/ro, grforce, kind='linear')
    grs_interp = np.geomspace(rs_thin[0]/ro, rs_thin[-1]/ro, 1001)
    ipot = potential.interpSphericalPotential(rforce_interp, 
        rgrid=rs_thin/ro, Phi0=pe_thin[0]/vo**2, ro=ro, vo=vo)

    # Plot
    trs = np.logspace(np.log10(rs.min()), np.log10(rs.max()), 100)
    fig = plt.figure()
    ax1 = fig.add_subplot(211)
    ax2 = fig.add_subplot(212)
    ax1.hist(np.log10(rs_star), bins=100, range=(np.log10(trs[0]), np.log10(trs[-1])), 
        log=True, histtype='step', edgecolor='Black')
    ax1.set_ylabel(r'$p(r_{\star})$')
    ax1.set_xlim(np.log10(trs[0]),np.log10(trs[-1]))
             
    ax2.plot(np.log10(rs_thin), np.log10(menc_thin), color='k', label='Enclosed Mass')
    ax2.plot(np.log10(trs), np.log10(ipot.mass(trs*apu.kpc).value), 
        color='Red', linestyle='dashed', label='Interpolated Mass')
    # ax2.set_xscale('log')
    # ax2.set_yscale('log')
    ax2.set_xlabel('log radius [kpc]')
    ax2.set_ylabel('enclosed mass [Msun]')
    ax2.set_xlim(np.log10(trs[0]),np.log10(trs[-1]))
    ax2.legend(loc='best')
    fig.savefig(fig_dir+str(z0_sid)+'_enclosed_mass.png',dpi=300)
    plt.close(fig)

    # Save the model
    ipot_filename = os.path.join(epsen_fitting_dir,
        str(z0_sid),'interp_potential.pkl')
    os.makedirs(os.path.dirname(ipot_filename),exist_ok=True)
    with open(ipot_filename,'wb') as handle:
        pickle.dump(ipot,handle)